In [7]:
from pathlib import Path
import lxml.etree as ET
import yaml
import os
import logging as log
from fastnumbers import fast_int

In [8]:
package_root = Path(".").absolute().parent.parent

In [9]:
f = Path(".").absolute().parent.parent / "datenbank_new.xml"
print(f)
assert f.is_file()

/home/klnrdknt/git/sephrasto-rework/datenbank_new.xml


In [10]:
tr = ET.parse(str(f)).getroot()
tr

<Element Datenbank at 0x7fd164af1680>

In [31]:
def xml_data_conversions(d: dict) -> dict:
    for label in [
        "kommentar",
        "linkKategorie",
        "steigerungsfaktor",
        "printclass",
        "kampffertigkeit",
        "typ",
        "rsBauch",
        "rsBeine",
        "rsBrust",
        "rsKopf",
        "rsLArm",
        "rsRArm",
        "system",
        "kosten",
        "variable",
        "verbilligt",
        "W6",
        "fk",
        "haerte",
        "plus",
        "wm",
        "lz",
        "rw",
    ]:
        if label in d:
            d[label] = fast_int(d[label], raise_on_invalid=True)
    if "attribute" in d:
        d["attribute"] = d["attribute"].split("|")
    if "voraussetzungen" in d:
        d["voraussetzungen"] = d["voraussetzungen"].split(" ODER ")
    if "kampfstile" in d:
        d["kampfstile"] = d["kampfstile"].split(", ")
    if "fertigkeiten" in d:
        d["fertigkeiten"] = d["fertigkeiten"].split(", ")
    return d

In [34]:
db_root = package_root / "db"
db_root.mkdir(exist_ok=True)
TAG_ATTRIBUTES = {}
for child in tr:
    # create folder
    tag = child.tag.replace("-", "_")
    folder = db_root / tag
    folder.mkdir(exist_ok=True)

    d = {k: v for k, v in sorted(child.items())}
    d["text"] = child.text
    d = xml_data_conversions(d)
    if tag in TAG_ATTRIBUTES:
        for k, v in d.items():
            TAG_ATTRIBUTES[tag][k] = v
    else:
        TAG_ATTRIBUTES[tag] = d

    try:
        filename = "_".join(
            d["name"]
            .lower()
            .replace("(", "")
            .replace(")", "")
            .replace("/", "_")
            .split(" ")
        )
    except KeyError:
        filename = "_".join(
            d["text"]
            .lower()
            .replace("(", "")
            .replace(")", "")
            .replace("/", "_")
            .split(" ")
        )
        # print(d)
    yaml_file = folder / f"{filename}.yaml"
    # if yaml_file.is_file():
    #    log.warning(f"yaml file {yaml_file} already exists")
    #    continue
    with open(yaml_file, "w+") as file:
        yaml.dump(d, file)

In [48]:
for k, v in TAG_ATTRIBUTES.items():
    s = f"{k}:\n"
    for l, w in v.items():
        s += (
            f"   {l}: {type(w)}\n".replace("<class 'int'>", str(-1))
            .replace("<class 'str'>", '""')
            .replace("<class 'list'>", '[""]')
            .replace("<class 'NoneType'>", '""')
        )
    with open(package_root / "templates" / "db_templates" / f"{k}.yaml", "w+") as file:
        file.write(s)

In [46]:
print(s)

FreieFertigkeit:
   kategorie: ""
   text: ""
   voraussetzungen: [""]



In [28]:
from sephrasto.template import character


def UI_from_list(l: list):
    pass


def UI_from_dict(d: dict, context: list = []):
    subwidgets = []
    for k, v in d.items():
        context += [k]
        if k.startswith("__"):
            continue
        if isinstance(v, dict):
            subwidgets.append(widgets.Label(k))
            subwidgets.append(UI_from_dict(v))
        elif isinstance(v, list):
            subwidgets.append(UI_from_list(v))
        elif isinstance(v, int):
            subwidgets.append(
                widgets.BoundedIntText(
                    value=v,
                    min=0,
                    step=1,
                    description=k if not k.startswith("_") else k.lstrip("_"),
                    disabled=k.startswith("_"),
                )
            )
        elif isinstance(v, str) and len(v) >= 50:
            subwidgets.append(widgets.HTML(description=k, placeholder=k, value=v))
        elif isinstance(v, str):
            subwidgets.append(
                widgets.Text(
                    value=v,
                    placeholder="Type something",
                    description=k if not k.startswith("_") else k.lstrip("_"),
                    disabled=k.startswith("_"),
                )
            )
    return widgets.VBox(subwidgets)


char_template = character()

UI_from_dict(char_template)

In [5]:
from ipywidgets import widgets

widgets.Text(description="hello")

Text(value='', description='hello')